<a href="https://colab.research.google.com/github/hamzarabi3/BTTS_prediction/blob/main/BTTS_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics  import accuracy_score
from joblib import dump, load
import numpy as np



*   Read data file
*   Convert datetime column
*   Create BTTS column



In [ ]:
df=pd.read_csv('data.csv')
df.date_GMT=pd.to_datetime(df['date_GMT'])
df['BTTS']=df.apply(lambda raw: 1 if raw.home_team_goal_count > 0 and raw.away_team_goal_count >0 else 0, axis=1)

In [ ]:
df.dropna(inplace=True)

* Team Filter

In [ ]:
# team='Chelsea'
# df=df.query('home_team_name == @team or away_team_name == @team').copy()

In [ ]:
df.shape

(2660, 24)

**Split data into train and test**

In [ ]:
train_df=df.query('date_GMT<"2018-08-10"')

test_df=df.query('date_GMT>="2018-08-10"')

In [ ]:
train_df.head(10)

,date_GMT,home_team_name,away_team_name,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,BTTS
0,2012-08-18 14:00:00,Arsenal,Sunderland,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,18.0,6.0,4.0,3.0,14.0,3.0,12.0,8.0,74.0,26.0,0
1,2012-08-18 14:00:00,Fulham,Norwich City,5.0,0.0,5.0,2.0,6.0,3.0,0.0,0.0,0.0,0.0,12.0,7.0,10.0,3.0,2.0,4.0,12.0,11.0,54.0,46.0,0
2,2012-08-18 14:00:00,Queens Park Rangers,Swansea City,0.0,5.0,5.0,1.0,5.0,3.0,2.0,0.0,2.0,0.0,13.0,12.0,5.0,7.0,8.0,5.0,12.0,15.0,50.0,50.0,0
3,2012-08-18 14:00:00,Reading,Stoke City,1.0,1.0,2.0,1.0,4.0,3.0,2.0,0.0,5.0,1.0,9.0,8.0,3.0,4.0,6.0,4.0,11.0,17.0,54.0,46.0,1
4,2012-08-18 14:00:00,West Bromwich Albion,Liverpool,3.0,0.0,3.0,1.0,7.0,3.0,1.0,0.0,4.0,1.0,13.0,10.0,7.0,3.0,6.0,7.0,11.0,12.0,42.0,58.0,0
5,2012-08-18 14:00:00,West Ham United,Aston Villa,1.0,0.0,1.0,1.0,6.0,4.0,0.0,0.0,1.0,0.0,8.0,8.0,4.0,3.0,4.0,5.0,17.0,9.0,33.0,67.0,0
6,2012-08-18 16:30:00,Newcastle United,Tottenham Hotspur,2.0,1.0,3.0,0.0,3.0,5.0,2.0,0.0,2.0,0.0,7.0,13.0,5.0,6.0,2.0,7.0,13.0,9.0,56.0,44.0,1
7,2012-08-19 12:30:00,Wigan Athletic,Chelsea,0.0,2.0,2.0,2.0,7.0,1.0,2.0,0.0,2.0,0.0,14.0,6.0,5.0,4.0,9.0,2.0,16.0,11.0,42.0,58.0,0
8,2012-08-19 15:00:00,Manchester City,Southampton,3.0,2.0,5.0,1.0,7.0,3.0,1.0,0.0,2.0,0.0,15.0,6.0,9.0,3.0,6.0,3.0,4.0,6.0,64.0,36.0,1
9,2012-08-20 19:00:00,Everton,Manchester United,1.0,0.0,1.0,0.0,6.0,8.0,1.0,0.0,2.0,0.0,17.0,13.0,8.0,8.0,9.0,5.0,12.0,11.0,42.0,58.0,0


## Create a torch dataset 

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import torch.nn as nn
from torch.optim import Adam
from sklearn.preprocessing import OneHotEncoder

***Train dataset***

In [ ]:
class TrainDataset(Dataset):
  def __init__(self,df,lags):
    self.nums=[ 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards', 'home_team_shots',
       'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_team_possession']
       
    df=df.reset_index()
    self.ohe=OneHotEncoder(handle_unknown='ignore', sparse=False)

    self.df_nums=df[self.nums]
    self.categ=['home_team_name', 'away_team_name']
    self.scaler=MinMaxScaler()
    self.df_nums=pd.DataFrame(self.scaler.fit_transform(self.df_nums),index=self.df_nums.index,columns=self.df_nums.columns)
    #use one hot encoder
    self.df_categ=df[self.categ]
    self.df_categ=pd.DataFrame(self.ohe.fit_transform(self.df_categ),index=self.df_categ.index)
    self.df_dates=df['date_GMT']
    self.label=df['BTTS']
    self.len=len(df)
    self.lags=lags
    self.df=df

  def save_scaler(self,file_name='minmax_scaler.joblib'):
    from joblib import dump
    dump(self.scaler,file_name)

  def save_encoder(self,file_name='ohe_encoder.joblib'):
    from joblib import dump
    dump(self.ohe,file_name)

  def __getitem__(self,ix):
    x1=torch.tensor(self.df_categ.iloc[ix])
    y=torch.tensor(self.label[ix])
    #step1 : create a placeholder tensor to put previous matches data
    x2=torch.zeros(lags,len(self.nums))
    date=self.df_dates[ix]
    current_play_day=f'{date.year}-{date.month}-{date.day}'

    #step2 : find the index of the last match played but not in the same day
    #because most matches played in the same day are programmed at the same hour, meaning we can't use them to predict.

    try:
      last_match_not_in_same_day_index  =  self.df[self.df.date_GMT<current_play_day].index[-1]
    except IndexError:
      #this is the first day of play
      last_match_not_in_same_day_index=None
    
    if last_match_not_in_same_day_index != None:
      farthest_lag_index  =  (last_match_not_in_same_day_index - self.lags) if (last_match_not_in_same_day_index - self.lags)>=0 else None
      if farthest_lag_index != None:
        x2=torch.tensor((self.df_nums.iloc[farthest_lag_index : last_match_not_in_same_day_index]).values)
      else:
        x2[lags-last_match_not_in_same_day_index-1:] =torch.tensor((self.df_nums.iloc[0 : last_match_not_in_same_day_index+1]).values) 

    x2=x2.reshape((-1,))
    x1=x1.reshape((-1,))
    x=torch.hstack([x1,x2])
    #return x.unsqueeze(dim=0),y.unsqueeze(dim=0)
    return x.float(),y.float()
  
  def __len__(self):
    return self.len

***Test dataset***

In [ ]:
class TestDataset(Dataset):
  def __init__(self,dataframe,lags,scaler,ohe,lags_tensor):
    self.nums=[ 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards', 'home_team_shots',
       'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_team_possession']
       
    dataframe=dataframe.reset_index()
    self.dataframe=dataframe
    self.lags_tensor=lags_tensor

    self.df_nums=dataframe[self.nums]
    self.categ=['home_team_name', 'away_team_name']

    #transform the data 
    
    self.scaler=scaler 
    self.ohe=ohe   
    self.df_nums=pd.DataFrame(self.scaler.transform(self.df_nums),index=self.df_nums.index,columns=self.df_nums.columns)
    self.df_categ=dataframe[self.categ]
    self.df_categ=pd.DataFrame(self.ohe.transform(self.df_categ),index=self.df_categ.index)
    self.df_dates=dataframe['date_GMT']
    self.label=dataframe['BTTS']
    self.len=len(dataframe)
    self.lags=lags

  def __getitem__(self,ix):
    x1=torch.tensor(self.df_categ.iloc[ix])
    y=torch.tensor(self.label[ix])
    #step1 : create a placeholder tensor to put previous matches data
    x2=self.lags_tensor
    date=self.df_dates[ix]
    current_play_day=f'{date.year}-{date.month}-{date.day}'

    #step2 : find the index of the last match played but not in the same day
    #because most matches played in the same day are programmed at the same hour, meaning we can't use them to predict.

    try:
      last_match_not_in_same_day_index  =  self.dataframe[self.dataframe.date_GMT<current_play_day].index[-1]
    except IndexError:
      #this is the first day of play
      last_match_not_in_same_day_index=None
    
    if last_match_not_in_same_day_index != None:
      farthest_lag_index  =  (last_match_not_in_same_day_index - self.lags) if (last_match_not_in_same_day_index - self.lags)>=0 else None
      if farthest_lag_index != None:
        x2=torch.tensor((self.df_nums.iloc[farthest_lag_index : last_match_not_in_same_day_index]).values)
      else:
        x2[lags-last_match_not_in_same_day_index-1:] =torch.tensor((self.df_nums.iloc[0 : last_match_not_in_same_day_index+1]).values) 

    x2=x2.reshape((-1,))
    x1=x1.reshape((-1,))
    x=torch.hstack([x1,x2])
    #return x.unsqueeze(dim=0),y.unsqueeze(dim=0)
    return x.float(),y.float()
  
  def __len__(self):
    return self.len

**Choose how many past matches to look at before making a prediction**

In [ ]:
lags=3
train_ds=TrainDataset(train_df,lags=lags)

* Save scaler of numerical features
* Save encoder for categorical features

In [ ]:
train_ds.save_scaler()
train_ds.save_encoder()

In [ ]:
x,y=train_ds[9]

In [ ]:
num_features=x.shape[0]

* For testing load the scaler and ecoder 
* Create the first lag tensor being the last one in the train dataset

In [ ]:
#for testing create the a lags_tensor for the first prediction
from joblib import load, dump


nums=[ 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards', 'home_team_shots',
       'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_team_possession']



mm_scaler=load('/content/minmax_scaler.joblib')
ohe_encoder=load('/content/ohe_encoder.joblib')

lags_tensor=train_df.iloc[-lags:][nums]
lags_tensor=torch.tensor(mm_scaler.transform(lags_tensor))

In [ ]:
test_ds=TestDataset(test_df,lags,mm_scaler,ohe_encoder,lags_tensor)

## Build the model

**Utility functions**

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
#device='cpu'

def get_model():
  model=nn.Sequential(
      
      nn.Linear(num_features,out_features=1)

  )
  
  lossfn=nn.BCEWithLogitsLoss()
  optim=Adam(model.parameters())
  return model.to(device), lossfn, optim

def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    
    return acc

In [ ]:
train_loader = DataLoader(train_ds,batch_size=64)
model, loss_fn, optimizer = get_model()

**Training loop**

In [ ]:
EPOCHS=15

model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = loss_fn(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.69557 | Acc: 0.490
Epoch 002: | Loss: 0.69398 | Acc: 0.500
Epoch 003: | Loss: 0.69331 | Acc: 0.505
Epoch 004: | Loss: 0.69261 | Acc: 0.511
Epoch 005: | Loss: 0.69195 | Acc: 0.515
Epoch 006: | Loss: 0.69131 | Acc: 0.516
Epoch 007: | Loss: 0.69070 | Acc: 0.518
Epoch 008: | Loss: 0.69011 | Acc: 0.522
Epoch 009: | Loss: 0.68955 | Acc: 0.537
Epoch 010: | Loss: 0.68902 | Acc: 0.539
Epoch 011: | Loss: 0.68850 | Acc: 0.541
Epoch 012: | Loss: 0.68801 | Acc: 0.540
Epoch 013: | Loss: 0.68754 | Acc: 0.544
Epoch 014: | Loss: 0.68709 | Acc: 0.545
Epoch 015: | Loss: 0.68665 | Acc: 0.548


**Testing the model**

In [ ]:
test_loader = DataLoader(test_ds,batch_size=64)

In [ ]:
x,y=test_ds[0]

In [ ]:
len(test_ds)

380

In [ ]:
accs = 0
preds=np.array([])
model.eval()
with torch.no_grad():
    for X_batch,y in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        preds=np.concatenate([preds,y_test_pred.cpu().numpy().flatten()])

In [ ]:
y=test_df.BTTS.values

In [ ]:
binary_acc(torch.tensor(preds),torch.tensor(y)).item()*100

52.63158082962036

In [ ]:
import warnings
import scipy
warnings.filterwarnings("ignore")
test_df.loc[:,'tempBTTS']=np.round(scipy.special.expit(preds))

In [ ]:
test_df.head(len(test_df))

,date_GMT,home_team_name,away_team_name,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,BTTS,tempBTTS
2282,2018-08-10 19:00:00,Manchester United,Leicester City,2.0,1.0,3.0,1.0,2.0,5.0,2.0,0.0,1.0,0.0,8.0,8.0,7.0,5.0,1.0,3.0,11.0,8.0,46.0,54.0,1,0.0
2283,2018-08-11 11:30:00,Newcastle United,Tottenham Hotspur,1.0,2.0,3.0,3.0,3.0,5.0,2.0,0.0,2.0,0.0,11.0,13.0,3.0,6.0,8.0,7.0,11.0,12.0,40.0,60.0,1,1.0
2284,2018-08-11 14:00:00,AFC Bournemouth,Cardiff City,2.0,0.0,2.0,1.0,7.0,4.0,1.0,0.0,1.0,0.0,11.0,7.0,5.0,2.0,6.0,5.0,11.0,9.0,62.0,38.0,0,0.0
2285,2018-08-11 14:00:00,Fulham,Crystal Palace,0.0,2.0,2.0,1.0,5.0,5.0,1.0,0.0,2.0,0.0,11.0,11.0,7.0,11.0,4.0,0.0,9.0,11.0,66.0,34.0,0,1.0
2286,2018-08-11 14:00:00,Huddersfield Town,Chelsea,0.0,3.0,3.0,2.0,2.0,5.0,2.0,0.0,1.0,0.0,6.0,8.0,2.0,5.0,4.0,3.0,9.0,8.0,37.0,63.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2657,2019-05-12 14:00:00,Liverpool,Wolverhampton Wanderers,2.0,0.0,2.0,1.0,4.0,1.0,0.0,0.0,2.0,0.0,11.0,7.0,6.0,3.0,5.0,4.0,3.0,11.0,59.0,41.0,0,1.0
2658,2019-05-12 14:00:00,Manchester United,Cardiff City,0.0,2.0,2.0,1.0,11.0,2.0,3.0,0.0,3.0,0.0,21.0,11.0,11.0,5.0,10.0,6.0,9.0,6.0,73.0,27.0,0,0.0
2659,2019-05-12 14:00:00,Southampton,Huddersfield Town,1.0,1.0,2.0,1.0,4.0,3.0,0.0,0.0,1.0,0.0,7.0,8.0,4.0,4.0,3.0,4.0,8.0,6.0,53.0,47.0,1,0.0
2660,2019-05-12 14:00:00,Tottenham Hotspur,Everton,2.0,2.0,4.0,1.0,7.0,4.0,0.0,0.0,2.0,0.0,9.0,13.0,4.0,10.0,5.0,3.0,10.0,13.0,55.0,45.0,1,1.0
